# Build file creation of cubes <img align="right" width="192" height="64" src="https://freemelt.com/app/uploads/freemeltLogo-1.png">

When developing parameters and settings for Electron Beam Manufacturing, it's often desirable to build several objects at once to shorten developing time. Usually, this is done by placing cubes in a grid, melting them once at a time. 

Importing the necessary libraries:

In [35]:
import obplib as obp
import os
import interactive_viewer as iw
import random as rand

In [36]:
class Cube:
    def __init__(self, midpoint, dimensions, speed, spot_size, power):
        self.midpoint = midpoint
        self.dimensions = dimensions
        self.speed = speed
        self.spot_size = spot_size
        self.power = power

In [37]:
%%add_to Cube

def _place_points(self, density):
    number_of_points = [int(self.dimensions[0] * density), int(self.dimensions[1] * density)]
    points = [
                [
                obp.Point(j / density + self.midpoint[0] - self.dimensions[0] / 2, i / density + self.midpoint[1] - self.dimensions[1] / 2) 
                for j in range(number_of_points[0] + 1)
                ]
                for i in range(number_of_points[1] + 1)
    ]
    return [ele for ele in points if ele] #return points within the build area

In [38]:
%%add_to Cube
def place_lines_one_direction(self, density, filename, reverse):
    # Generate the points on the powder bed
    points = self._place_points(density)

    # Create the lines based on the points
    lines = []
    y_range = range(len(points))
    bp = obp.Beamparameters(self.spot_size, self.power)
    for y in y_range:
        lines.append(obp.Line(points[y][0], points[y][-1], self.speed, bp))

    # Write the lines to the file
    try:
        old_obp = obp.read_obp(filename)
        new_obp = old_obp + lines
        obp.write_obp(new_obp, filename)
    except:
        obp.write_obp(lines, filename)
        
    if reverse == True:
        lines = [obp.Line(line.P2, line.P1, line.Speed, line.bp) for line in reversed(lines)]

    return lines

In [39]:
%%add_to Cube
#calls the melt pattern on each island in a random order
def melt_cube(self, density, filename, reverse = False):
    lines = cube.place_lines_one_direction(density, filename, reverse) #change for different melting pattern
    return lines

## Example - 3$\times$3 cubes

In [40]:
#code example 
filename = "hej3.obp"
try:    
    os.remove(filename)
except:
    pass

density = 0.1
rec_numbers = [3, 3]
dimensions = [100, 100]
midpoint_list = [
    [
        (i + 1 - rec_numbers[0]/2) * dimensions[0] * 2, 
        (j + 1 - rec_numbers[1]/2) * dimensions[1] * 2
    ] 
    for i in range(rec_numbers[0]) for j in range(rec_numbers[1])
]
cube_list = [
    Cube(midpoint = midpoint_list[i], dimensions=[100, 100], speed=100, spot_size=100, power=100)
        for i in range(rec_numbers[0] * rec_numbers[1])
]
lines = []
for i in range(rec_numbers[0] * rec_numbers[1]):
    lines = lines + cube_list[i].place_lines_one_direction(density, filename, False)

iw.interactive_viewer(lines)

interactive(children=(IntText(value=99, description='Max Lines:'), IntSlider(value=50, description='upper_lim'…

The strategy of melting one cube at a time can lead to too much heat being unloaded into one cube. To spread out the heat, randomizing the lines will lead to the following pattern:

In [41]:
rand.shuffle(lines)
iw.interactive_viewer(lines)

interactive(children=(IntText(value=99, description='Max Lines:'), IntSlider(value=50, description='upper_lim'…